# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [9]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [10]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [11]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [12]:
stp1 = data.groupby(["CustomerID", "ProductID", "ProductName"]).agg({"Quantity": "sum"})

stp1.head()

Quantity
CustomerID ProductID ProductName                              
33         23        Crab - Imitation Flakes                 1
           51        Wine - Crozes Hermitage E.              1
           53        Cassis                                  1
           67        Wine - Sogrape Mateus Rose              1
           70        Squid - Tubes / Tenticles 10/20         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [13]:
pivot = pd.pivot_table(stp1, 
                       index="ProductName",
                       values="Quantity", 
                       columns="CustomerID",
                      fill_value=0)

pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [14]:
distances = pdist(pivot.T, "euclidean")

customerMatrix = pd.DataFrame(1/(1+squareform(distances)), index=pivot.columns, columns=pivot.columns)

customerMatrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [15]:
# Para el cliente con el ID 200...
#cojo desde el segundo elemento porque el primero es el mismo nº

similarMatrix = customerMatrix[200].sort_values(ascending=False)[1:6]
similarMatrix = similarMatrix.index.tolist()

similarMatrix

[3317, 1072, 3535, 1920, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [16]:
stp5 = stp1.loc[similarMatrix]

stp5.head()

Quantity
CustomerID ProductID ProductName                          
1072       13        Water, Tap                          1
           22        Rice - Jasmine Sented               1
           28        Sobe - Tropical Energy              1
           29        Spinach - Baby                      1
           30        Cheese - Brie, Triple Creme         1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [17]:
stp6 = stp5.groupby("ProductName").agg({"Quantity": "sum"}).reset_index().sort_values(by="Quantity",
                                                                                      ascending=False)

display(stp6.head())


,ProductName,Quantity
159,Soup - Campbells Bean Medley,4
103,Muffin - Carrot Individual Wrap,3
8,Bay Leaf,3
136,Pork - Kidney,3
129,"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [18]:
# Merge del df con los productos más comprados por los clientes más parecidos a 200
# + la matriz que relaciona cliente por producto 

ranked = stp6.copy()

stp7 = ranked.merge(pivot, how='left', on='ProductName') 
#stp7 = ranked.merge(stp1.loc[200], how='left', on='ProductName')

stp7.head()

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Soup - Campbells Bean Medley,4,0,0,1,0,0,0,2,0,...,0,0,0,25,50,25,0,0,0,0
1,Muffin - Carrot Individual Wrap,3,0,0,0,0,0,0,0,0,...,0,0,0,0,25,0,0,0,0,0
2,Bay Leaf,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
3,Pork - Kidney,3,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,0,0,0
4,"Pepper - Black, Whole",3,1,1,0,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [19]:
sinComprar = stp7[stp7[200] == 0]

sinComprar.head()

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Soup - Campbells Bean Medley,4,0,0,1,0,0,0,2,0,...,0,0,0,25,50,25,0,0,0,0
1,Muffin - Carrot Individual Wrap,3,0,0,0,0,0,0,0,0,...,0,0,0,0,25,0,0,0,0,0
2,Bay Leaf,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
3,Pork - Kidney,3,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,0,0,0
6,Wanton Wrap,3,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
recomendaciones = sinComprar["ProductName"][:5].tolist()

recomendaciones

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [31]:
todasRecomendaciones = {}

clientes = data["CustomerID"].unique().tolist()

In [35]:
def recommendations(matrix, dct):
    for e in clientes:
        similarMatrix = matrix[e].sort_values(ascending=False)[1:6].index.tolist() #5 clientes más parecidos
        similarRecords = stp1.loc[similarMatrix] #todo lo que compran esos clientes
        purchases = similarRecords.groupby("ProductName").agg({"Quantity": "sum"}).reset_index().sort_values(by="Quantity",
                                                                                          ascending=False) #lo que + compran los clientes
        stp7 = purchases.merge(pivot, how='left', on='ProductName') 
        sinComprar = stp7[stp7[200] == 0]
        recomendaciones = sinComprar["ProductName"][:5].tolist()
        dct[e] = recomendaciones    
    return dct

        

In [36]:
recommendations(customerMatrix, todasRecomendaciones);

In [42]:
list(todasRecomendaciones.items())[5]

(83106,
 ['Cocoa Butter',
  'Cheese - Boursin, Garlic / Herbs',
  'Cake - Mini Cheesecake',
  'Beef - Top Sirloin - Aaa',
  'Cheese - Taleggio D.o.p.'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [27]:
stp9 = pd.DataFrame(todasRecomendaciones).T

columns = ["Product {}".format(i) for i in range(1,6)]

stp9.head()

,0,1,2,3,4
61288,Beef - Rib Eye Aaa,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,"Lamb - Pieces, Diced"
77352,Wine - Valpolicella Masi,Fuji Apples,Ocean Spray - Ruby Red,Tuna - Salad Premix,Chicken - Soup Base
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,"Cheese - Brie,danish",Cookie - Dough Variety,Tuna - Salad Premix
23548,Tea - English Breakfast,Squid - Tubes / Tenticles 10/20,Spinach - Baby,Jolt Cola - Electric Blue,Sprouts - Baby Pea Tendrils
78981,Isomalt,Vanilla Beans,Lettuce - Frisee,Grouper - Fresh,Corn Meal


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [26]:
distances2 = pdist(pivot.T, "cosine")

customerMatrix2 = pd.DataFrame(1/(1+squareform(distances)), index=pivot.columns, columns=pivot.columns)


In [38]:
todasRecomendaciones2 = {}

recommendations(customerMatrix2, todasRecomendaciones2);

In [43]:
list(todasRecomendaciones2.items())[5]

(83106,
 ['Cocoa Butter',
  'Cheese - Boursin, Garlic / Herbs',
  'Cake - Mini Cheesecake',
  'Beef - Top Sirloin - Aaa',
  'Cheese - Taleggio D.o.p.'])